# **Разведывательный анализ данных**

Перед тем как строить модель для кредитного скоринга, нам нужно провести разведывательный анализ данных, чтобы узнать больше о данных содержащихся в датасете.

Выделим несколько целей, которые мы бы хотели достичь в результате анализа данных:

1. Узнать о смысле признаков для нашей модели, посмотреть на их распределение, попопробовать на начальном этапе откинуть несколько заведамо неинформативных признаков, выделить целевую переменную;
2. Посмотреть, есть ли пропуски в нашем датасете, если же есть, то проанализировать стоит ли выкидывать объекты с пропусками, или заменить их какими-либо точечными статистиками (mean, median и т.д.);
3. Преобразовать признаки, если требуется их преобразовать;
4. Предложить варианты решения исходной задачи, построить baseline модели машинного обучения, выделить ключевые метрики для оптимизации модели. 


In [3]:
#Подключим библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
credit_score_df = pd.read_csv("~/Documents/datasets/application_train.csv")

credit_score_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


Для начала нам нужно исследовать наш датасет [Kaggle: Home Credit Default Risk](https://www.kaggle.com/c/home-credit-default-risk/data), данный датасет использовался для соренования на платформе Kaggle, участники данного соревнования решали задачу классификации того, вернет ли заёмщик деньги банку или нет (для кредиторов без кредитной истории), то есть решали задачу кредитного скоринга, соревнование проходило в 2018 году.

Датасет содержит 122 призника, но этих признаков слишком много для создания большой модели в условниях ограниченных вычислительных ресурсов, поэтому для создания модели и анализа данных мы используем только первые 41 признаков, приведём описание данных призаков:

- SK_ID_CURR - ID кредита в выборке;
- TARGET - Целевая переменная (1 - клиент, испытывающий трудности с оплатой: он просрочил платеж более чем на X дней по крайней мере по одному из первых Y платежей по кредиту в нашей выборке, 0 - все остальные случаи);
- NAME_CONTRACT_TYPE - Идентификатор, является ли кредит наличным или возобновляемым;
- CODE_GENDER - Пол клиента;
- FLAG_OWN_CAR - Идентификатор, если клиент владеет автомобилем;
- FLAG_OWN_REALTY - Идентификатор, владеет ли клиент домом или квартирой;
- CNT_CHILDREN - Количество детей у клиента;
- AMT_INCOME_TOTAL - Доход клиента;
- AMT_CREDIT - Сумма кредита;
- AMT_ANNUITY - Кредитный аннуитет;
- AMT_GOODS_PRICE - Для потребительских кредитов - это цена товара, на который выдается кредит;
- NAME_TYPE_SUITE - Кто сопровождал клиента при оформлении кредита;
- NAME_INCOME_TYPE - Тип дохода клиента (бизнесмен, работающий, находящийся в декретном отпуске и т.д.);
- NAME_EDUCATION_TYPE - Уровень высшего образования, полученного клиентом;
- NAME_FAMILY_STATUS - Семейное положение клиента;
- NAME_HOUSING_TYPE - Какова жилищная ситуация клиента (снимает жилье, живет с родителями и т.д.);
- REGION_POPULATION_RELATIVE - Нормализованная численность населения региона, в котором живет клиент (большее число означает, что клиент живет в более населенном регионе);
- DAYS_BIRTH - Возраст клиента в днях на момент подачи заявки;
- DAYS_EMPLOYED - За сколько дней до подачи заявления человек приступил к текущей работе;
- DAYS_REGISTRATION - За сколько дней до подачи заявки клиент изменил свою регистрацию
- DAYS_ID_PUBLISH - За сколько дней до подачи заявки клиент сменил документ, удостоверяющий личность, с которым он подавал заявку на кредит;
- OWN_CAR_AGE - Возраст автомобиля клиента;
- FLAG_MOBIL - Предоставил ли клиент мобильный телефон (1=ДА, 0=НЕТ);
- FLAG_EMP_PHONE - Предоставил ли клиент рабочий телефон (1=ДА, 0=НЕТ);
- FLAG_WORK_PHONE - Предоставил ли клиент рабочий телефон (1=ДА, 0=НЕТ);
- FLAG_CONT_MOBILE - Был ли мобильный телефон доступен (1=ДА, 0=НЕТ);
- FLAG_PHONE - Предоставил ли клиент домашний телефон (1=ДА, 0=НЕТ);
- FLAG_EMAIL - Предоставил ли клиент электронную почту (1=ДА, 0=НЕТ);
- OCCUPATION_TYPE - Какой род занятий у клиента;
- CNT_FAM_MEMBERS - Сколько членов семьи у клиента;
- REGION_RATING_CLIENT - Наш рейтинг региона, в котором живет клиент (1,2,3);
- REGION_RATING_CLIENT_W_CITY - Наш рейтинг региона проживания клиента с учетом города (1,2,3);
- WEEKDAY_APPR_PROCESS_START - В какой день недели клиент подал заявку на получение кредита;
- HOUR_APPR_PROCESS_START - Примерно в котором часу клиент подал заявку на получение кредита;
- REG_REGION_NOT_LIVE_REGION - Идентификатор, если постоянный адрес клиента не совпадает с контактным адресом (1=разный, 0=одинаковый, на уровне региона);
- REG_REGION_NOT_WORK_REGION - Идентификатор, если постоянный адрес клиента не совпадает с рабочим адресом (1=разный, 0=одинаковый, на уровне региона);
- LIVE_REGION_NOT_WORK_REGION - Идентификатор, если контактный адрес клиента не совпадает с рабочим адресом (1=разный, 0=одинаковый, на уровне региона);
- REG_CITY_NOT_LIVE_CITY - Идентификатор, если постоянный адрес клиента не совпадает с контактным адресом (1=разный, 0=одинаковый, на уровне города);
- REG_CITY_NOT_WORK_CITY - Идентификатор, если постоянный адрес клиента не совпадает с адресом работы (1=разный, 0=одинаковый, на уровне города);
- LIVE_CITY_NOT_WORK_CITY - Идентификатор, если контактный адрес клиента не совпадает с рабочим адресом (1=разный, 0=одинаковый, на уровне города);
- ORGANIZATION_TYPE - Тип организации, в которой работает клиент.

In [10]:
#Уменьшим количество признаков
credit_score_df = credit_score_df.iloc[:,:41]
credit_score_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,1,MONDAY,11,0,0,0,0,0,0,School
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,2,MONDAY,9,0,0,0,0,0,0,Government
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,2,WEDNESDAY,17,0,0,0,0,0,0,Business Entity Type 3
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,2,THURSDAY,11,0,0,0,0,1,1,Religion


In [11]:
#Удалим незначащий признак, отвечающий за ID кредита
credit_score_df.drop(columns=["SK_ID_CURR"], inplace = True)